In [ ]:
#%matplotlib qt

In [ ]:
import numpy as np
import os
import mne
import yasa
import math
import matplotlib as plt

In [ ]:
participants = ['BPM0004','BPM0005','BPM0006','BPM0007','BPM0008','BPM0009','BPM0010','BPM0011','BPM0012','BPM0013','BPM0014','BPM0015','BPM0016','BPM0017','BPM0018','BPM0019','BPM0020','BPM0021','BPM0022','BPM0023','BPM0024','BPM0025','BPM0026','BPM0027','BPM0028','BPM0029','BPM0032','BPM0033','BPM0034','BPM0035','BPM0036','BPM0038','BPM0040','BPM0041','BPM0042','BPM0044','BPM0045','BPM0046','BPM0047','BPM0048','BPM0049','BPM0050','BPM0051','BPM0052','BPM0053','BPM0054','BPM0055','BPM0056','BPM0057','BPM0058','BPM0059','BPM0060','BPM0061','BPM0062','BPM0063','BPM0064','BPM0065','BPM0066','BPM0067','BPM0068','BPM0069','BPM0070','BPM0071','BPM0072','BPM0073','BPM0074','BPM0075','BPM0076','BPM0077','BPM0078','BPM0079','BPM0080','BPM0081','BPM0082','BPM0083','BPM0084','BPM0085','BPM0088','BPM0089','BPM0090','BPM0091','BPM0092','BPM0093','BPM0094','BPM0095','BPM0096','BPM0097','BPM0098','BPM0099','BPM0100','BPM0101','BPM0102','BPM0103','BPM0104','BPM0105','BPM0106','BPM0107','BPM0108','BPM0109','BPM0110','BPM0111','BPM0112','BPM0113','BPM0114','BPM0115','BPM0117','BPM0118','BPM0119','BPM0120','BPM0121','BPM0122','BPM0123','BPM0124','BPM0125','BPM0126','BPM0127','BPM0128','BPM0129','BPM0130','BPM0131','BPM0132','BPM0133','BPM0134','BPM0135','BPM0136','BPM0137','BPM0138','BPM0139','BPM0141','BPM0142','BPM0143','BPM0148','BPM0149','BPM0153','BPM0154','BPM0155','BPM0158','BPM0161','BPM0162','BPM0165','BPM0167','BPM0169','BPM0171','BPM0175','BPM0176','BPM0177','BPM0178','BPM0179','BPM0181','BPM0182','BPM0183','BPM0185','BPM0186','BPM0188','BPM0192','BPM0193','BPM0194','BPM0195','BPM0196','BPM0197','BPM0198','BPM0202','BPM0205','BPM0208','BPM0209','BPM0210','BPM0211','BPM0212','BPM0213','BPM0214','BPM0215','BPM0216','BPM0220','BPM0221','BPM0224','BPM0225','BPM0226','BPM0227','BPM0228''BPM0229','BPM0230','BPM0231','BPM0233','BPM0234','BPM0235','BPM0236','BPM0237','BPM0238','BPM0239','BPM0240','BPM0241','BPM0242','BPM0243','BPM0244','BPM0245','BPM0246','BPM0247','BPM0248','BPM0249','BPM0250','BPM0251']
# Select window size
window = 2

# Initialize large matrices or lists to store results for all participants
all_results_filt = []
wrong_amplitude_channels = []

In [ ]:
# Define the REM detection function that creates the matrix content
def detect_rems(raw_data, roc_channel, loc_channel, hypno, artefacts):
    """Detect REMs using YASA's REM detection function.

    Args:
        raw_data: Raw data object from MNE.
        roc_channel: Name of the ROC channel.
        loc_channel: Name of the LOC channel.

    Returns:
        REM detection results.
    """
    # Get the sampling frequency
    sf = raw_data.info['sfreq']

    # Given your sampling frequency is 512 Hz, each 2-second window will have 512 * 2 = 1024 samples.
    window_size = window * sf  # 2-second windows
    num_windows = raw_data.n_times // window_size

    # Placeholder for results
    rem_window_results = []

    # Create copies of the raw data for each channel
    raw_copy_roc = raw_data.copy().pick([roc_channel])
    raw_copy_loc = raw_data.copy().pick([loc_channel])

    # Get the data for each channel
    #ROC_data = raw_copy_roc.get_data(units="mV")[0]  # Select first (and only) channel's data
    #LOC_data = raw_copy_loc.get_data(units="mV")[0]  # Select first (and only) channel's data
    ROC_data = raw_copy_roc.get_data(units="uV")[0]  # Select first (and only) channel's data
    LOC_data = raw_copy_loc.get_data(units="uV")[0]  # Select first (and only) channel's data

    # After scaling, try running rem_detect again
    rem = yasa.rem_detect(ROC_data, LOC_data, sf=sf, hypno=None)

    # If no REM events are detected, try with millivolts
    if rem is None or rem.summary().empty:
        # Need to skip these channels
        ROC_data = raw_copy_roc.get_data(units="mV")[0]  # Select first (and only) channel's data
        LOC_data = raw_copy_loc.get_data(units="mV")[0]  # Select first (and only) channel's data
        wrong_amplitude_channels.append(participant)

        rem = yasa.rem_detect(ROC_data, LOC_data, sf=sf, hypno=None)

     # If still no REM events, skip this participant
    if rem is None or rem.summary().empty:
        wrong_amplitude_channels.append(participant)  # Log this participant
        rem_window_results.append((participant, 0, 0, 0, 0, 0, 0, 0))
        return 0, sf, ROC_data, LOC_data, rem_window_results
    else:
        # Extract the summary of REM events
        rem_events = rem.summary()

        # Iterate over each 2-second window
        for i in range(int(num_windows)*2):
            # Adjust the start and end times for each window
            start_sec = i  # Start from 1 sec and then add window * i
            end_sec = start_sec + window
            hypno_value = hypno[i // 20]  # Get hypnogram value for the current 2-sec window

            rem_in_window = rem_events[(rem_events['Peak'] >= start_sec) & (rem_events['Peak'] < end_sec)]
            loc_max_amplitude = max(ROC_data[int(start_sec * sf):int(end_sec * sf)])
            roc_max_amplitude = max(LOC_data[int(start_sec * sf):int(end_sec * sf)])
            window_max_amplitude = max(loc_max_amplitude, roc_max_amplitude)

            artifact_value = artefacts[i // 4]
            if not rem_in_window.empty:
                max_loc_abs_peak = rem_in_window['LOCAbsValPeak'].max()
                rem_window_results.append((participant, start_sec, end_sec, artifact_value,  hypno_value, True, window_max_amplitude, max_loc_abs_peak))
            else:
                max_loc_abs_peak = 0
                rem_window_results.append((participant, start_sec, end_sec, artifact_value, hypno_value, False, window_max_amplitude, max_loc_abs_peak))

        return rem, sf, ROC_data, LOC_data, rem_window_results

# Define the function to save the data
def save_to_txt(file_path, data):
    # Convert list of tuples to a NumPy array for saving
    data_array = np.array(data, dtype=object)

    # Save the array to a text file
    np.savetxt(file_path, data_array, fmt='%s', delimiter=',')

def save_to_txt(data, file_path):
    # Convert list of tuples to a NumPy array for saving
    data_array = np.array(data, dtype=object)

    # Check if the file_path is a string and if data_array is correctly formatted
    if isinstance(file_path, str) and data_array.ndim == 2:
        # Save the array to a text file
        np.savetxt(file_path, data_array, fmt='%s', delimiter=',')
    else:
        print("Error: file_path must be a string and data must be a 2D array")


In [ ]:
for participant in participants:

    edf_path = fr'D:\EDF\{participant}.edf'
    hypnogram_path = fr'D:\hypno\{participant}.hyp.txt'
    artefact_path = fr'D:\automatic_artefacts\{participant}.art.txt'
    print("######################Current participant: ", participant, "########################")

    #Load the participants EDF file
    raw_data = mne.io.read_raw_edf(edf_path, preload=True, include=['REOG', 'LEOG', 'T1/A1A2', 'T2/A1A2','eog1', 'eog2', 'eog1-a2', 'eog2-a1', 'LOC - ROC', 'EOG1+'])

    # Load participant's hypnogram
    # Load the hypnogram file (had to rename it because of the "." and the " ")
    hypno = np.loadtxt(hypnogram_path)

    # Calculate the duration of the raw data in seconds
    raw_data_duration_sec = raw_data.times[-1]

    # Calculate the expected number of hypnogram data points
    expected_hypno_length = math.ceil(raw_data_duration_sec / 20)

    # Compare the actual hypnogram length with the expected length
    if len(hypno) == expected_hypno_length:
        print("The hypnogram file matches the length of the raw_data file.")
    else:
        print(f"Mismatch in lengths. Expected {expected_hypno_length} data points in the hypnogram, found {len(hypno)}.")

    # Check if there is a mismatch in lengths
    if len(hypno) < expected_hypno_length:
        missing_length = expected_hypno_length - len(hypno)
        print(f"Mismatch in lengths. Expected {expected_hypno_length} data points in the artefact file, found {len(hypno)}. Appending {missing_length} missing rows.")

        # Create an array of ones for the missing data points
        missing_data = np.ones(missing_length)

        # Append the missing data to the artefacts array
        hypno = np.concatenate((hypno, missing_data))
    else:
        print("The hypnogram file matches the length of the raw_data file.")

    
    # Calculate the expected number of hypnogram data points
    expected_artefact_length = math.ceil(raw_data_duration_sec / 4)
    # Try to load participant's artefact data
    try:
        artefacts = np.loadtxt(artefact_path)
    except FileNotFoundError:
        print(f"No artefact file found for {participant}. Using an array of ones.")
        # Calculate the expected number of artefact data points
        artefacts = np.ones(expected_artefact_length)  # Create an array of ones

    # Compare the actual hypnogram length with the expected length
    if len(artefacts) == expected_artefact_length:
        print("The artefact file matches the length of the raw_data file.")
    else:
        print(f"Mismatch in lengths. Expected {expected_artefact_length} data points in the artefact file, found {len(artefacts)}.")

    # Check if there is a mismatch in lengths
    if len(artefacts) < expected_artefact_length:
        missing_length = expected_artefact_length - len(artefacts)
        print(f"Mismatch in lengths. Expected {expected_artefact_length} data points in the artefact file, found {len(artefacts)}. Appending {missing_length} missing rows.")

        # Create an array of ones for the missing data points
        missing_data = np.ones(missing_length)

        # Append the missing data to the artefacts array
        artefacts = np.concatenate((artefacts, missing_data))
    else:
        print("The artefact file matches the length of the raw_data file.")



    channel_names = raw_data.info['ch_names']

    # Define potential ROC-LOC channel pairs based on common configurations in your dataset
    channel_pairs = [
        ('REOG', 'LEOG'),
        ('T1/A1A2', 'T2/A1A2'),
        ('eog1-a2', 'eog2-a1'),
        ('eog1', 'eog2'),
        #('ROC - A1, LOC - A2')
        ]

    detected = False

    # Check if EOG1+ is in channel names and proceed with the specific logic for this case
    print(channel_names)
    if 'EOG1+' or 'LOC - ROC' in channel_names:
        ch_name = raw_data.info['ch_names'][0]
        # Logic for EOG1+ case
        eog_plus_data = raw_data.pick_channels([ch_name]).get_data()
        eog_neg_data = -1 * eog_plus_data
        ch_info = mne.create_info(ch_names=['EOG-'], sfreq=raw_data.info['sfreq'], ch_types=['eog'])
        eog_neg_raw = mne.io.RawArray(eog_neg_data, ch_info)
        raw_data.add_channels([eog_neg_raw], force_update_info=True)
        # Define custom scaling for EOG channels
        # scalings = {'eog': 20e-6}  # This is an example scaling factor in Volts
        # Create a copy of the raw data
        raw_data_filtered = raw_data.copy()
        # Apply a bandpass filter between 0.3 and 10 Hz to the data (focus just EOG+/-)
        raw_data_filtered.filter(l_freq=0.3, h_freq=10, picks=[ch_name, 'EOG-'])
        # Call the detect_rems function for both unfiltered and filtered data
        # rem_unfilt_eog, sf_unfilt, roc_unfilt_eog, loc_unfilt_eog, rem_window_results_unfilt_eog = detect_rems(raw_data, 'EOG1+', 'EOG-', hypno, artefacts)
        rem_filt_eog, sf_filt, roc_filt_eog, loc_filt_eog, rem_window_results_filt_eog = detect_rems(raw_data_filtered, ch_name, 'EOG-', hypno, artefacts)
        detected = True
    else:
        # Check for any of the other EOG channel pairs
        for roc, loc in channel_pairs:
            if roc in channel_names and loc in channel_names:
                print(roc + loc)
                # Define custom scaling for EOG channels
                # scalings = {'eog': 20e-6}  # This is an example scaling factor in Volts
                # Create a copy of the raw data
                raw_data_filtered = raw_data.copy()
                # Apply a bandpass filter between 0.3 and 10 Hz to the data (focus just EOG+/-)
                # raw_data_filtered.filter(l_freq=0.3, h_freq=10, picks=['EOG+','ROC-M1','LOC-M2','EOG-'])
                raw_data_filtered.filter(l_freq=0.3, h_freq=10, picks=[roc,loc])
                # Call the detect_rems function for both unfiltered and filtered data
                #####rem_unfilt_eog, sf_unfilt, roc_unfilt_eog, loc_unfilt_eog, rem_window_results_unfilt_eog = detect_rems(raw_data, roc, loc, hypno, artefacts)
                rem_filt_eog, sf_filt, roc_filt_eog, loc_filt_eog, rem_window_results_filt_eog = detect_rems(raw_data_filtered, roc, loc, hypno, artefacts)
                # ROC-LOC pair found, proceed with REM detection using these channels
                detected = True
                #break  # Exit the loop once a matching pair is found and processed

    if not detected:
        print("Required EOG channels not found in the dataset.")



    # Append the results to the all_results lists
    # all_results_unfilt.extend(rem_window_results_unfilt_eog)
    all_results_filt.extend(rem_window_results_filt_eog)

# Convert lists to NumPy arrays for efficient storage (if necessary)
# all_results_unfilt_array = np.array(all_results_unfilt)
all_results_filt_array = np.array(all_results_filt)
output_path_filt =  r'C:\Users\Webpredict\Documents\MSCthesis\all_results_filt.txt'
save_to_txt(all_results_filt,output_path_filt)